# -----------------------------------
# 목차
## Step 1. 데이터 다운로드
## Step 2. 데이터 읽어오기
## Step 3. 데이터 정제
## Step 4. 평가 데이터셋 분리
## Step 5. 인공지능 만들기
# -----------------------------------

## Step 1. 데이터 다운로드
### Cloud shell에 아래의 명령어 입력하기
### mkdir -p ~/aiffel/lyricist/models
###  ln -s ~/data ~/aiffel/lyricist/data

## Step 2. 데이터 읽어오기¶

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/Exploration/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## Step 3. 데이터 정제

### shakespeare.txt의 경우, txt파일을 열어보면
### - First Citizen:
### - Before we proceed any further, hear me speak.
### -  
### - All:
### - Speak, speak.
### 이렇게 되어 있으나 lyrics 파일의 경우, 예를 들어 adele.txt 파일을 열어보면
### - Looking for some education
### - Made my way into the night
### - All that bullshit conversation
### - 이렇게 되어 있기 때문에 '문장의 끝이 :'로 끝나거나 '공백'인 문장을 제외할 필요가 없음

In [2]:
import re 
import numpy as np
import tensorflow as tf

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고, 양쪽 공백을 없애기
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백 넣기
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개의 공백을 하나의 공백으로 바꾸기
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾸기
    sentence = sentence.strip() # 다시 양쪽 공백을 지우기
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에는 <start>, 끝에는 <end>를 추가하기
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [3]:
corpus = []

for sentence in raw_corpus:
    # 원치 않는 문장 넘기기
    if len(sentence) == 0: continue
    # 정제하고 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    
    if len((preprocessed_sentence).split()) > 15 : continue #토큰 15개 넘는 것은 제외하기
    
    
    corpus.append(preprocessed_sentence)
        
# 정제된 결과 10개만 확인하기
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [4]:
def tokenize(corpus):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, # 단어장 크기 12,000개 이상
        filters=' ',
        oov_token="<unk>"
    )

    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2971 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  117 ...    0    0    0]
 [   2  258  195 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fd3c03c12b0>


In [5]:
print(tensor[:3, :10])

[[   2   50    4   95  303   62   53    9  946 6269]
 [   2   15 2971  872    5    8   11 5747    6  374]
 [   2   33    7   40   16  164  288   28  333    5]]


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6269    3    0    0    0]
[  50    4   95  303   62   53    9  946 6269    3    0    0    0    0]


# Step 4. 평가 데이터셋 분리

In [8]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.2, 
                                                    random_state=7)
print('enc_train 개수: ', len(enc_train),', dec_train 개수: ', len(dec_train))

enc_train 개수:  124981 , dec_train 개수:  124981


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# Step 5. 인공지능 만들기

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
# 데이터 한 배치만 불러오기.
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-8.57947598e-05,  7.78614194e-05, -1.95558780e-04, ...,
         -5.97379876e-05, -9.27443980e-05,  9.59210593e-05],
        [ 2.55299441e-04,  3.67578410e-04,  1.40814700e-05, ...,
         -4.80417584e-05, -1.31288383e-04,  9.36184442e-05],
        [ 1.98625115e-04,  6.37353631e-04,  2.56465573e-04, ...,
         -1.04803643e-04, -3.13693134e-04,  9.51179172e-05],
        ...,
        [-1.52474199e-03,  8.56081897e-04, -5.21429640e-04, ...,
          1.01702230e-03, -9.59177443e-04,  8.17397784e-04],
        [-1.55256141e-03,  9.48905945e-04, -8.19268287e-04, ...,
          1.09451299e-03, -9.63981322e-04,  1.04702159e-03],
        [-1.53897342e-03,  1.04870705e-03, -1.07510388e-03, ...,
          1.15974073e-03, -9.64918756e-04,  1.26489787e-03]],

       [[-8.57947598e-05,  7.78614194e-05, -1.95558780e-04, ...,
         -5.97379876e-05, -9.27443980e-05,  9.59210593e-05],
        [ 1.35312730e-04,  3.15372192e-04, -3

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  18466976  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  32780704  
_________________________________________________________________
dense (Dense)                multiple                  24302025  
Total params: 78,621,961
Trainable params: 78,621,961
Non-trainable params: 0
_________________________________________________________________


In [13]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=4, validation_data=(enc_val,dec_val), batch_size=16)

Epoch 1/4
7812/7812 [==============================] - 596s 76ms/step - loss: 2.9207 - val_loss: 2.6786
Epoch 2/4
7812/7812 [==============================] - 598s 77ms/step - loss: 2.4262 - val_loss: 2.4650
Epoch 3/4
7812/7812 [==============================] - 599s 77ms/step - loss: 2.0402 - val_loss: 2.3470
Epoch 4/4
7812/7812 [==============================] - 599s 77ms/step - loss: 1.7286 - val_loss: 2.2988


In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    

    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [15]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so , <end> '

# REPROT
### 훈련을 시키는데 너무 많은 시간이 걸려서 많이 돌려보지 못해서 아쉽다.
### 처음에 만들었을때 val_loss가 나오지 않아서 당황스러웠는데 코드를 덜 작성한 탓이었다..
### 이후에 '거울아 거울아~' 이 노드를 보면서 참고해서 수정을 했다.
### 이번에는 아람님의 도움을 받았다.
### 학습을 하다가 val_loss가 2.3까지 내려갔다가 계속 훈련하니 2.4로 다시 올라가는 경우가 있었는데, 이 부분은 batch_size=16로 줄이고, epochs=4로 줄이는 것으로 해결되었다.
### i love you so 라는 문장도 해석해보면 전혀 이상한 문장도 아니라서 이번 프로젝트도 잘 해결되었다! (찾아보니 해당 제목의 노래도 있었다!)